# MLP multi-classe con Softmax/Cross-Entropy (Iris)

> Esercitazione (student).

## Iris dataset + preprocessing

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

data=load_iris()
X=data['data']; y=data['target']
Xtr,Xte,ytr,yte=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
sc=StandardScaler().fit(Xtr)
Xtr=sc.transform(Xtr); Xte=sc.transform(Xte)
print(Xtr.shape,Xte.shape)

## MLP PyTorch (logits -> CrossEntropyLoss)

In [ ]:
try:
 import torch, torch.nn as nn
 Xt=torch.tensor(Xtr,dtype=torch.float32)
 yt=torch.tensor(ytr,dtype=torch.long)
 Xv=torch.tensor(Xte,dtype=torch.float32)
 yv=torch.tensor(yte,dtype=torch.long)
 model=nn.Sequential(nn.Linear(4,16), nn.ReLU(), nn.Linear(16,3))
 opt=torch.optim.Adam(model.parameters(), lr=0.01)
 loss_fn=nn.CrossEntropyLoss()
 for _ in range(400):
  opt.zero_grad(); logits=model(Xt); loss=loss_fn(logits,yt); loss.backward(); opt.step()
 with torch.no_grad():
  pred=model(Xv).argmax(1)
  acc=(pred==yv).float().mean().item()
 print('acc', acc)
except Exception as e:
 print('Torch non disponibile:', e)

## Esercizio: dropout + confusion matrix

In [ ]:
# TODO